# LAB 2 Naive Bayes
Nov. 7th, 2024
by Tavares h. and Cole M.

In [ ]:
import numpy as np
import pandas as pd

carData = pd.read_csv('motor_vehicles.csv')
print(carData.shape)  # gives a tuple with the shape of data frame

In [2]:
print(carData.head())

  fuel_efficiency maintenance doors    type   size safety RATING
0       excellent       vhigh   two  sports  small    low   JUNK
1       excellent       vhigh   two  sports  small    med   JUNK
2       excellent       vhigh   two  sports  small   high   JUNK
3       excellent       vhigh   two  sports    med    low   JUNK
4       excellent       vhigh   two  sports    med    med   JUNK


In [3]:
splitRatio = .75

In [4]:
fuel_labels = ['crap', 'med', 'good', 'excellent']
maint_labels = ['low', 'med', 'high', 'vhigh']
door_labels = ['two', 'three', 'four','many']
type_labels = ['sports', 'sedan', 'wagon']
size_labels = ['small', 'med', 'big']
safety_labels = ['low', 'med', 'high']

In [5]:
# Do the shuffle!
reCarData = carData.reindex(np.random.permutation(carData.index))
shuffCarData = reCarData.reset_index()

In [6]:
shuffCarData

,index,fuel_efficiency,maintenance,doors,type,size,safety,RATING
0,390,excellent,low,four,sedan,med,low,JUNK
1,46,excellent,vhigh,three,wagon,small,med,JUNK
2,798,good,low,three,sedan,big,low,JUNK
3,1356,crap,vhigh,four,sports,big,low,JUNK
4,201,excellent,high,many,sedan,med,low,JUNK
...,...,...,...,...,...,...,...,...
1723,215,excellent,high,many,wagon,big,high,JUNK
1724,1510,crap,high,many,wagon,big,med,OKAY
1725,706,good,med,four,sports,med,med,JUNK
1726,207,excellent,high,many,wagon,small,low,JUNK


In [7]:
shuffCarData.pop('index')

0        390
1         46
2        798
3       1356
4        201
        ... 
1723     215
1724    1510
1725     706
1726     207
1727    1710
Name: index, Length: 1728, dtype: int64

In [8]:
shuffCarData

,fuel_efficiency,maintenance,doors,type,size,safety,RATING
0,excellent,low,four,sedan,med,low,JUNK
1,excellent,vhigh,three,wagon,small,med,JUNK
2,good,low,three,sedan,big,low,JUNK
3,crap,vhigh,four,sports,big,low,JUNK
4,excellent,high,many,sedan,med,low,JUNK
...,...,...,...,...,...,...,...
1723,excellent,high,many,wagon,big,high,JUNK
1724,crap,high,many,wagon,big,med,OKAY
1725,good,med,four,sports,med,med,JUNK
1726,excellent,high,many,wagon,small,low,JUNK


In [9]:
# function definition: getting the counts (frequencies of occurrence)
def count(data,colname,label,target):    # for whichever data set, pick a column and one value in that column
                                         # and an output label (target) to compare against
    condition = (data[colname] == label) & (data['RATING'] == target)
    print(data[condition])
    return len(data[condition])

In [10]:
# list of predicted values
predicted = []

# dictionary to store probabilities
probabilities = {0:{},1:{}}

#length of current training set
train_len = int(splitRatio * len(shuffCarData))
print(train_len)

#splitting the data into training and test sets
training = shuffCarData.iloc[:train_len,:] # the first train_len rows, all columns

1296


In [11]:
# debugging step: looking at the training set
training

,fuel_efficiency,maintenance,doors,type,size,safety,RATING
0,excellent,low,four,sedan,med,low,JUNK
1,excellent,vhigh,three,wagon,small,med,JUNK
2,good,low,three,sedan,big,low,JUNK
3,crap,vhigh,four,sports,big,low,JUNK
4,excellent,high,many,sedan,med,low,JUNK
...,...,...,...,...,...,...,...
1291,excellent,low,two,sports,med,med,JUNK
1292,excellent,med,three,sports,big,med,JUNK
1293,good,high,two,sedan,med,med,JUNK
1294,good,vhigh,two,sports,small,high,JUNK


In [12]:
test_X = shuffCarData.iloc[train_len:,:-1] # features in the test set
test_y = shuffCarData.iloc[train_len:,-1]  # targets (the last entry in each row; i.e., the last column)

In [13]:
# debugging
print(test_X)

     fuel_efficiency maintenance  doors    type   size safety
1296            good        high   many   wagon    med    med
1297            good        high   four  sports    med    med
1298            good        high   many   sedan    med   high
1299            good         low  three   wagon  small    med
1300            crap       vhigh  three   wagon  small   high
...              ...         ...    ...     ...    ...    ...
1723       excellent        high   many   wagon    big   high
1724            crap        high   many   wagon    big    med
1725            good         med   four  sports    med    med
1726       excellent        high   many   wagon  small    low
1727            crap         low   many   sedan  small    low

[432 rows x 6 columns]


In [14]:
# debugging
print(test_y)

1296    OKAY
1297    JUNK
1298    OKAY
1299    JUNK
1300    OKAY
        ... 
1723    JUNK
1724    OKAY
1725    JUNK
1726    JUNK
1727    JUNK
Name: RATING, Length: 432, dtype: object


In [15]:
# checking for dimension match
print(test_X.shape)
print(test_y.shape)

(432, 6)
(432,)


In [16]:
# counts of prior 'LOW' and 'HIGH' labels (targets)
count_LOW = count(training, 'RATING', 'JUNK', 'JUNK')
count_HIGH = count(training, 'RATING', 'OKAY', 'OKAY')

prior_LOW = count_LOW / len(training)
prior_HIGH = count_HIGH / len(training)

print('The prior \'JUNK\' count is', count_LOW)
print('and \'OKAY\',',count_HIGH, '.')
print('The prior probability of \'JUNK\' is', prior_LOW)
print('and of \'OKAY\',', prior_HIGH, '.')

     fuel_efficiency maintenance  doors    type   size safety RATING
0          excellent         low   four   sedan    med    low   JUNK
1          excellent       vhigh  three   wagon  small    med   JUNK
2               good         low  three   sedan    big    low   JUNK
3               crap       vhigh   four  sports    big    low   JUNK
4          excellent        high   many   sedan    med    low   JUNK
...              ...         ...    ...     ...    ...    ...    ...
1291       excellent         low    two  sports    med    med   JUNK
1292       excellent         med  three  sports    big    med   JUNK
1293            good        high    two   sedan    med    med   JUNK
1294            good       vhigh    two  sports  small   high   JUNK
1295       excellent         med   four   sedan  small    low   JUNK

[904 rows x 7 columns]
     fuel_efficiency maintenance  doors   type   size safety RATING
20              crap       vhigh  three  sedan    big   high   OKAY
32          

In [17]:
# TRAINING: learning the priors and class-conditionals from the training set
for col in training.columns[:-1]:
    probabilities[0][col] = {}
    probabilities[1][col] = {}

    if col == "fuel_efficiency":
        for k in fuel_labels:
            count_k_LOW = count(training, col, k, 'JUNK')
            count_k_HIGH = count(training, col, k, 'OKAY')
            probabilities[0][col][k] = count_k_LOW / count_LOW
            probabilities[1][col][k] = count_k_HIGH / count_HIGH
            probabilities[0][col][k] += 0.001 # throwing in a quick-and-dirty approximation of the
            probabilities[1][col][k] += 0.001 # "m estimate" for avoiding zeros in the product    
    else:
        if col == "maintenance":
            for k in maint_labels:
                count_k_LOW = count(training, col, k, 'JUNK')
                count_k_HIGH = count(training, col, k, 'OKAY')
                probabilities[0][col][k] = count_k_LOW / count_LOW
                probabilities[1][col][k] = count_k_HIGH / count_HIGH
                probabilities[0][col][k] += 0.001 # throwing in a quick-and-dirty approximation of the
                probabilities[1][col][k] += 0.001 # "m estimate" for avoiding zeros in the product
        else:
            if col == "doors":
                for k in door_labels:
                    count_k_LOW = count(training, col, k, 'JUNK')
                    count_k_HIGH = count(training, col, k, 'OKAY')
                    probabilities[0][col][k] = count_k_LOW / count_LOW
                    probabilities[1][col][k] = count_k_HIGH / count_HIGH
                    probabilities[0][col][k] += 0.001 # throwing in a quick-and-dirty approximation of the
                    probabilities[1][col][k] += 0.001 # "m estimate" for avoiding zeros in the product
            else:
                if col == "type":
                    for k in type_labels:
                        count_k_LOW = count(training, col, k, 'JUNK')
                        count_k_HIGH = count(training, col, k, 'OKAY')
                        probabilities[0][col][k] = count_k_LOW / count_LOW
                        probabilities[1][col][k] = count_k_HIGH / count_HIGH
                        probabilities[0][col][k] += 0.001 # throwing in a quick-and-dirty approximation of the
                        probabilities[1][col][k] += 0.001 # "m estimate" for avoiding zeros in the product
                else:
                    if col == "size":
                        for k in size_labels:
                            count_k_LOW = count(training, col, k, 'JUNK')
                            count_k_HIGH = count(training, col, k, 'OKAY')
                            probabilities[0][col][k] = count_k_LOW / count_LOW
                            probabilities[1][col][k] = count_k_HIGH / count_HIGH
                            probabilities[0][col][k] += 0.001 # throwing in a quick-and-dirty approximation of the
                            probabilities[1][col][k] += 0.001 # "m estimate" for avoiding zeros in the product
                    else:
                        for k in safety_labels:
                            count_k_LOW = count(training, col, k, 'JUNK')
                            count_k_HIGH = count(training, col, k, 'OKAY')
                            probabilities[0][col][k] = count_k_LOW / count_LOW
                            probabilities[1][col][k] = count_k_HIGH / count_HIGH
                            probabilities[0][col][k] += 0.001 # throwing in a quick-and-dirty approximation of the
                            probabilities[1][col][k] += 0.001 # "m estimate" for avoiding zeros in the product
                
                
            

     fuel_efficiency maintenance  doors    type   size safety RATING
3               crap       vhigh   four  sports    big    low   JUNK
7               crap         med   many   sedan  small    low   JUNK
13              crap        high   four   sedan  small    low   JUNK
15              crap         med  three   sedan    big    low   JUNK
17              crap         low  three  sports    big    med   JUNK
...              ...         ...    ...     ...    ...    ...    ...
1262            crap        high    two  sports    med    low   JUNK
1271            crap         med  three  sports  small    med   JUNK
1272            crap         low    two  sports    med    med   JUNK
1275            crap         med   many  sports    med    med   JUNK
1288            crap         low  three   sedan    big    low   JUNK

[185 rows x 7 columns]
     fuel_efficiency maintenance  doors   type   size safety RATING
20              crap       vhigh  three  sedan    big   high   OKAY
52          

In [18]:
import pprint

# Sample dictionary for demonstration
pp = pprint.PrettyPrinter(indent=4)

# Format each value in the nested dictionaries and print
formatted_probabilities = {
    key: {inner_key: {sub_key: "{:.3f}".format(sub_value) for sub_key, sub_value in inner_value.items()}
           for inner_key, inner_value in value.items()}
    for key, value in probabilities.items()
}

pp.pprint(formatted_probabilities)

{   0: {   'doors': {   'four': '0.243',
                        'many': '0.228',
                        'three': '0.245',
                        'two': '0.288'},
           'fuel_efficiency': {   'crap': '0.206',
                                  'excellent': '0.303',
                                  'good': '0.269',
                                  'med': '0.227'},
           'maintenance': {   'high': '0.253',
                              'low': '0.219',
                              'med': '0.238',
                              'vhigh': '0.294'},
           'safety': {'high': '0.229', 'low': '0.483', 'med': '0.291'},
           'size': {'big': '0.301', 'med': '0.326', 'small': '0.376'},
           'type': {'sedan': '0.252', 'sports': '0.489', 'wagon': '0.262'}},
    1: {   'doors': {   'four': '0.267',
                        'many': '0.243',
                        'three': '0.288',
                        'two': '0.205'},
           'fuel_efficiency': {   'crap': '0.219',
  

In [19]:
# TESTING: reading rows from the test set, checking the value of each feature and comparing with the target outcome
for row in range(len(test_X)):
    prod_LOW = prior_LOW
    prod_HIGH = prior_HIGH
        
    for feature in test_X.columns:
        # Check if the feature value exists in the probabilities dictionary:
        if test_X[feature].iloc[row] in probabilities[0][feature]:
            prod_LOW *= probabilities[0][feature][test_X[feature].iloc[row]]
        else:
            prod_LOW *= 0  # or some default value if the feature value is not found

        if test_X[feature].iloc[row] in probabilities[1][feature]:
            prod_HIGH *= probabilities[1][feature][test_X[feature].iloc[row]]
        else:
            prod_HIGH *= 0  # or some default value if the feature value is not found
            
    # predicting the outcome
    if prod_LOW > prod_HIGH:
        predicted.append('JUNK')
    else:
        predicted.append('OKAY')

# Initialize correct and incorrect counts outside the loop
correct = 0
incorrect = 0

for j in range(len(test_y)):
    if predicted[j] == 'JUNK':
        if test_y.iloc[j] == 'JUNK':
            correct += 1
        else:
            incorrect += 1
    else:
        if test_y.iloc[j] == 'JUNK':
            incorrect += 1  # This should be incorrect since predicted is 'HIGH'
        else:
            correct += 1  # This should be correct since predicted is 'HIGH'

print('The accuracy is', correct / (correct + incorrect))

The accuracy is 0.8819444444444444
